# Connect to Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd /gdrive/My Drive/AN2DL-challenge-2022-polimi

/gdrive/My Drive/AN2DL-challenge-2022-polimi


# Import libraries

In [3]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16) 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import interp1d
import warnings
import logging

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.9.2


# Set seed for reproducibility

In [4]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Data preprocessing

In [5]:
x_all = np.load("Dataset/training_dataset_homework2/x_train.npy")
y_all = np.load("Dataset/training_dataset_homework2/y_train.npy")

In [6]:
x_all.shape, y_all.shape

((2429, 36, 6), (2429,))

In [7]:
# Convert the sparse labels to categorical values
y_all = tfk.utils.to_categorical(y_all)
y_all.shape

(2429, 12)

In [8]:
x_all.shape[1]

36

Interpolation

In [9]:
def interp_on_axis(x_all, m, axis):
  x = np.arange(x_all.shape[axis])

  # Create a linear interpolator.
  f = interp1d(x, x_all, axis=axis)

  xx = np.linspace(x[0], x[-1], (x_all.shape[axis] - 1)*(m + 1) + 1)
  b = f(xx)

  return b

In [10]:
x_interp = interp_on_axis(x_all, 3, 1)

In [11]:
x_interp.shape

(2429, 141, 6)

Z-axis editing

In [12]:
x_more_features = interp_on_axis(x_all, 1, 2)

In [13]:
x_more_features.shape

(2429, 36, 11)

In [14]:
x_interp_more_features = interp_on_axis(x_more_features, 1, 1)
x_interp_more_features.shape

(2429, 71, 11)

Splitting

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_interp_more_features, y_all, test_size=0.2)

In [16]:
x_train.shape

(1943, 71, 11)

In [17]:
y_train.shape

(1943, 12)

Class weighting

In [18]:
category_weight = {}
elements_per_class = {}
classes = 12

for i in range(classes):
    category_weight[i] = 0.0

for i in range(classes):
    elements_per_class[i] = 0

for elem_class in y_train:
    for i in range(classes):
      if elem_class[i]==1.:
          elements_per_class[i]+=1

total_elements = len(y_train)

for i in category_weight.keys():
    category_weight[i] = total_elements / (classes * elements_per_class[i])

# Build and train models 

In [46]:
input_shape = x_train.shape[1:]
classes = y_train.shape[-1]
batch_size = 256
epochs = 3000

## 1DCNN_classifier

In [56]:
keras.backend.clear_session()

In [57]:
from tensorflow.keras import regularizers

def build_1DCNN_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    noise = tfkl.GaussianNoise(0.4, seed = seed)(input_layer)
    cnn = tfkl.Conv1D(256,3,padding='same',activation='relu')(noise)
    cnn = tfkl.MaxPooling1D()(cnn)
    cnn = tfkl.Conv1D(256,3,padding='same',activation='relu')(cnn)
    dropout = tfkl.Dropout(.3, seed=seed)(cnn)
    cnn = tfkl.Conv1D(256,3,padding='same',activation='relu')(dropout)
    gap = tfkl.GlobalAveragePooling1D()(cnn)
    dropout = tfkl.Dropout(.4, seed=seed)(gap)

    # Classifier
    classifier = tfkl.Dense(320, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5))(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False), metrics='accuracy')

    # Return the model
    return model

In [58]:
model_1DCNN = build_1DCNN_classifier(input_shape, classes)
model_1DCNN.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 71, 11)]          0         
                                                                 
 gaussian_noise (GaussianNoi  (None, 71, 11)           0         
 se)                                                             
                                                                 
 conv1d (Conv1D)             (None, 71, 256)           8704      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 35, 256)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 35, 256)           196864    
                                                                 
 dropout (Dropout)           (None, 35, 256)           0     

In [59]:
history = model_1DCNN.fit(
    x = x_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=(x_val, y_val),
    class_weight = category_weight,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=300, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=140, factor=0.5, min_lr=1e-5)
    ]
).history

Epoch 1/3000
8/8 [==============================] - 1s 54ms/step - loss: 23.0239 - accuracy: 0.0561 - val_loss: 4.6382 - val_accuracy: 0.1770 - lr: 0.0010
Epoch 2/3000
8/8 [==============================] - 0s 23ms/step - loss: 12.7485 - accuracy: 0.0762 - val_loss: 4.1418 - val_accuracy: 0.0720 - lr: 0.0010
Epoch 3/3000
8/8 [==============================] - 0s 21ms/step - loss: 6.8425 - accuracy: 0.0885 - val_loss: 3.8037 - val_accuracy: 0.0700 - lr: 0.0010
Epoch 4/3000
8/8 [==============================] - 0s 17ms/step - loss: 5.0440 - accuracy: 0.0808 - val_loss: 3.1917 - val_accuracy: 0.1111 - lr: 0.0010
Epoch 5/3000
8/8 [==============================] - 0s 17ms/step - loss: 3.7161 - accuracy: 0.1060 - val_loss: 2.8903 - val_accuracy: 0.0926 - lr: 0.0010
Epoch 6/3000
8/8 [==============================] - 0s 18ms/step - loss: 4.3362 - accuracy: 0.1050 - val_loss: 3.0103 - val_accuracy: 0.1049 - lr: 0.0010
Epoch 7/3000
8/8 [==============================] - 0s 20ms/step - loss: 2

In [60]:
model_1DCNN.save("2012_1DCNN_Nico")

In [52]:
del model_1DCNN

## BiLSTM classifier

In [63]:
def build_BiLSTM_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    bilstm = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True))(input_layer)
    dropout = tfkl.Dropout(.4, seed=seed)(bilstm)
    bilstm = tfkl.Bidirectional(tfkl.LSTM(128))(dropout)
    dropout = tfkl.Dropout(.5, seed=seed)(bilstm)

    # Classifier
    classifier = tfkl.Dense(320, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5))(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False), metrics='accuracy')

    # Return the model
    return model

In [64]:
model_BiLSTM = build_BiLSTM_classifier(input_shape, classes)
model_BiLSTM.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 71, 11)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 71, 256)          143360    
 l)                                                              
                                                                 
 dropout_2 (Dropout)         (None, 71, 256)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 320)               82240 

In [65]:
history = model_BiLSTM.fit(
    x = x_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=(x_val, y_val),
    class_weight = category_weight,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=300, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=140, factor=0.5, min_lr=1e-5)
    ]
).history

Epoch 1/3000
8/8 [==============================] - 8s 231ms/step - loss: 2.4611 - accuracy: 0.0906 - val_loss: 2.3987 - val_accuracy: 0.1399 - lr: 0.0010
Epoch 2/3000
8/8 [==============================] - 0s 41ms/step - loss: 2.1368 - accuracy: 0.2141 - val_loss: 2.1135 - val_accuracy: 0.2469 - lr: 0.0010
Epoch 3/3000
8/8 [==============================] - 0s 41ms/step - loss: 1.8241 - accuracy: 0.2717 - val_loss: 1.9068 - val_accuracy: 0.2449 - lr: 0.0010
Epoch 4/3000
8/8 [==============================] - 0s 41ms/step - loss: 1.5832 - accuracy: 0.3248 - val_loss: 1.8987 - val_accuracy: 0.2428 - lr: 0.0010
Epoch 5/3000
8/8 [==============================] - 0s 41ms/step - loss: 1.4210 - accuracy: 0.3572 - val_loss: 1.7449 - val_accuracy: 0.2963 - lr: 0.0010
Epoch 6/3000
8/8 [==============================] - 0s 42ms/step - loss: 1.3076 - accuracy: 0.3603 - val_loss: 1.6978 - val_accuracy: 0.3395 - lr: 0.0010
Epoch 7/3000
8/8 [==============================] - 0s 42ms/step - loss: 1.

In [ ]:
model_BiLSTM.save("2012_BiLSTM_Nico")

## RESNET classifier

In [66]:
def build_RESNET_model(input_shape, nb_classes):
        n_feature_maps = 64

        input_layer = tfkl.Input(input_shape)

        # BLOCK 1

        conv_x = tfkl.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = tfkl.BatchNormalization()(conv_x)
        conv_x = tfkl.Activation('relu')(conv_x)

        conv_y = tfkl.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
        conv_y = tfkl.BatchNormalization()(conv_y)
        conv_y = tfkl.Activation('relu')(conv_y)

        conv_z = tfkl.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = tfkl.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = tfkl.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = tfkl.BatchNormalization()(shortcut_y)

        output_block_1 = tfkl.add([shortcut_y, conv_z])
        output_block_1 = tfkl.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
        conv_x = tfkl.BatchNormalization()(conv_x)
        conv_x = tfkl.Activation('relu')(conv_x)

        conv_y = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = tfkl.BatchNormalization()(conv_y)
        conv_y = tfkl.Activation('relu')(conv_y)

        conv_z = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = tfkl.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = tfkl.BatchNormalization()(shortcut_y)

        output_block_2 = tfkl.add([shortcut_y, conv_z])
        output_block_2 = tfkl.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = tfkl.BatchNormalization()(conv_x)
        conv_x = tfkl.Activation('relu')(conv_x)

        conv_y = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = tfkl.BatchNormalization()(conv_y)
        conv_y = tfkl.Activation('relu')(conv_y)

        conv_z = tfkl.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = tfkl.BatchNormalization()(conv_z)

        # no need to expand channels because they are equal
        shortcut_y = tfkl.BatchNormalization()(output_block_2)

        output_block_3 = tfkl.add([shortcut_y, conv_z])
        output_block_3 = tfkl.Activation('relu')(output_block_3)

        # FINAL

        gap_layer = tfkl.GlobalAveragePooling1D()(output_block_3)

        output_layer = tfkl.Dense(nb_classes, activation='softmax')(gap_layer)

        model = tfk.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=tfk.optimizers.Adam(),
                      metrics=['accuracy'], sample_weight_mode='temporal')
        
        return model

In [67]:
model_resnet = build_RESNET_model(input_shape, classes)

In [68]:
history = model_resnet.fit(
    x = x_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=(x_val, y_val),
    class_weight = category_weight,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=300, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=140, factor=0.5, min_lr=1e-5)
    ]
).history

Epoch 1/3000
8/8 [==============================] - 8s 575ms/step - loss: 2.5962 - accuracy: 0.1400 - val_loss: 3.7419 - val_accuracy: 0.0432 - lr: 0.0010
Epoch 2/3000
8/8 [==============================] - 0s 49ms/step - loss: 2.1560 - accuracy: 0.1714 - val_loss: 3.2990 - val_accuracy: 0.0597 - lr: 0.0010
Epoch 3/3000
8/8 [==============================] - 0s 49ms/step - loss: 1.9642 - accuracy: 0.2434 - val_loss: 2.6881 - val_accuracy: 0.1523 - lr: 0.0010
Epoch 4/3000
8/8 [==============================] - 0s 49ms/step - loss: 1.8295 - accuracy: 0.3052 - val_loss: 2.5014 - val_accuracy: 0.2798 - lr: 0.0010
Epoch 5/3000
8/8 [==============================] - 0s 49ms/step - loss: 1.6525 - accuracy: 0.2898 - val_loss: 2.2337 - val_accuracy: 0.4609 - lr: 0.0010
Epoch 6/3000
8/8 [==============================] - 0s 47ms/step - loss: 1.5263 - accuracy: 0.3196 - val_loss: 2.1673 - val_accuracy: 0.2058 - lr: 0.0010
Epoch 7/3000
8/8 [==============================] - 0s 47ms/step - loss: 1.

## GRU classifier

In [71]:
def build_GRU_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    noise = tfkl.GaussianNoise(.4, seed=seed)(input_layer)
    gru = tfkl.Bidirectional(tfkl.GRU(256, return_sequences=True))(noise)
    dropout = tfkl.Dropout(.3, seed=seed)(gru)
    gru = tfkl.Bidirectional(tfkl.GRU(256))(dropout)
    dropout = tfkl.Dropout(.4, seed=seed)(gru)

    # Classifier
    classifier = tfkl.Dense(320, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5))(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False), metrics='accuracy')

    # Return the model
    return model

In [72]:
model_GRU = build_GRU_classifier(input_shape, classes)

In [73]:
history = model_GRU.fit(
    x = x_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=(x_val, y_val),
    class_weight = category_weight,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=300, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=140, factor=0.5, min_lr=1e-5)
    ]
).history

Epoch 1/3000
8/8 [==============================] - 7s 262ms/step - loss: 2.4523 - accuracy: 0.1230 - val_loss: 2.2811 - val_accuracy: 0.1893 - lr: 0.0010
Epoch 2/3000
8/8 [==============================] - 1s 90ms/step - loss: 1.9209 - accuracy: 0.2584 - val_loss: 2.0558 - val_accuracy: 0.2613 - lr: 0.0010
Epoch 3/3000
8/8 [==============================] - 1s 89ms/step - loss: 1.6987 - accuracy: 0.3134 - val_loss: 1.9775 - val_accuracy: 0.2654 - lr: 0.0010
Epoch 4/3000
8/8 [==============================] - 1s 90ms/step - loss: 1.4775 - accuracy: 0.3567 - val_loss: 1.9250 - val_accuracy: 0.2840 - lr: 0.0010
Epoch 5/3000
8/8 [==============================] - 1s 90ms/step - loss: 1.3242 - accuracy: 0.4189 - val_loss: 1.8151 - val_accuracy: 0.3416 - lr: 0.0010
Epoch 6/3000
8/8 [==============================] - 1s 87ms/step - loss: 1.1823 - accuracy: 0.4334 - val_loss: 1.7805 - val_accuracy: 0.3251 - lr: 0.0010
Epoch 7/3000
8/8 [==============================] - 1s 90ms/step - loss: 1.

In [ ]:
del model_GRU